<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [14]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [15]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
try:
    with open('../../.venv/.secret/hopsworks-api-key.txt', 'r') as file:
        os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
except:
    pass
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1180820
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "switzerland", "city": "lucerne", "street": "adligenswilerstrasse", "aqicn_url": "https://api.waqi.info/feed/A75070", "latitude": 47.05, "longitude": 8.31}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [26]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [17]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,0.0,switzerland,lucerne,adligenswilerstrasse,2024-11-19,https://api.waqi.info/feed/A75070


In [19]:
aq_today_df.info(), air_quality_fg.insert(aq_today_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 176.0+ bytes
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1180820/fs/1171523/fg/1355112


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:09 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1180820/jobs/named/air_quality_1_offline_fg_materialization/executions


(None,
 (<hsfs.core.job.Job at 0x74498c626de0>, {
    "results": [
      {
        "expectation_config": {
          "expectation_type": "expect_column_min_to_be_between",
          "meta": {
            "expectationId": 678939
          },
          "kwargs": {
            "column": "pm25",
            "min_value": -0.1,
            "max_value": 500.0,
            "strict_min": true
          }
        },
        "success": true,
        "result": {
          "observed_value": 0.0,
          "element_count": 1,
          "missing_count": null,
          "missing_percent": null
        },
        "exception_info": {
          "raised_exception": false,
          "exception_message": null,
          "exception_traceback": null
        },
        "meta": {
          "ingestionResult": "INGESTED",
          "validationTime": "2024-11-19T08:14:03.000315Z"
        }
      }
    ],
    "success": true,
    "meta": {
      "great_expectations_version": "0.15.12",
      "expectation_suite_name

## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [54]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 47.25°N 8.5°E
Elevation 436.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-19,9.20,0.1,14.982228,215.217682,lucerne
1,2024-11-20,4.25,0.1,15.542662,256.607483,lucerne
2,2024-11-21,3.30,0.0,5.959060,64.983147,lucerne
3,2024-11-22,-0.10,0.0,17.253731,246.644409,lucerne
4,2024-11-23,1.95,0.0,0.804985,296.564972,lucerne
5,2024-11-24,7.55,0.0,3.319036,192.528793,lucerne
6,2024-11-25,8.05,0.0,1.527351,224.999893,lucerne
7,2024-11-26,7.95,0.0,1.484318,165.963730,lucerne
8,2024-11-27,5.15,1.0,9.826088,8.426887,lucerne
9,2024-11-28,5.70,0.0,3.075841,290.556122,lucerne


In [50]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 452.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [22]:
# Insert new data
# air_quality_fg.insert(aq_today_df)

Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1180820/fs/1171523/fg/1355112


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:09 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1180820/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x74498c626de0>,
 {
   "results": [
     {
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "meta": {
           "expectationId": 678939
         },
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         }
       },
       "success": true,
       "result": {
         "observed_value": 0.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-19T08:14:56.000848Z"
       }
     }
   ],
   "success": true,
   "meta": {
     "great_expectations_version": "0.15.12",
     "expectation_suite_name": "aq_expectation_suite",
     "run_id": {

In [71]:
today_ = datetime.datetime.now() - datetime.timedelta(10)
df_rolling: pd.DataFrame = air_quality_fg.filter(air_quality_fg.date >= today_).read()[['date', 'pm25']]
# sort df_rolling by date
df_rolling = df_rolling.sort_values(by='date').rolling(window=3, on='date').mean()
df_rolling['date'] = df_rolling['date'] + datetime.timedelta(1)
df_rolling = df_rolling.rename(columns={"pm25": "pm25_rolling"})
df_rolling['date'] = df_rolling['date'].dt.tz_localize(None)
df_rolling.dropna(inplace=True)
df_rolling

Finished: Reading data from Hopsworks, using ArrowFlight (0.79s) 


,date,pm25_rolling
6,2024-11-13,2.360000
4,2024-11-14,2.233333
3,2024-11-15,1.990000
5,2024-11-16,2.403333
0,2024-11-17,2.396667
2,2024-11-18,2.456667
7,2024-11-19,1.926667
9,2024-11-20,1.093333


In [72]:
weather_fg.filter(weather_fg.date >= today_).read()

Finished: Reading data from Hopsworks, using ArrowFlight (0.82s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,pm25_rolling
0,2024-11-13 00:00:00+00:00,4.737916,1.2,7.596341,19.584793,lucerne,2.360000
1,2024-11-15 00:00:00+00:00,3.269167,0.0,3.710795,293.983429,lucerne,1.990000
2,2024-11-17 00:00:00+00:00,5.333750,0.4,7.100310,27.216135,lucerne,2.396667
3,2024-11-10 00:00:00+00:00,7.227499,0.0,3.710795,349.464783,lucerne,3.416667
4,2024-11-14 00:00:00+00:00,4.065000,0.0,6.112053,355.924774,lucerne,2.233333
5,2024-11-12 00:00:00+00:00,6.335834,3.4,12.744457,35.100521,lucerne,3.133333
6,2024-11-11 00:00:00+00:00,7.452501,6.7,8.019601,345.304749,lucerne,3.383333
7,2024-11-16 00:00:00+00:00,3.840000,0.0,4.044997,330.107605,lucerne,2.403333


In [75]:
earliest_aq_date = pd.Series.min(df_rolling['date'])
earliest_aq_date = earliest_aq_date.strftime('%Y-%m-%d')
earliest_aq_date

weather_df = util.get_historical_weather(city, earliest_aq_date, str(today), latitude, longitude)

# join on date with rolling mean
weather_df = weather_df.merge(df_rolling, on='date', how='left')
# remove rows with NaN values
weather_df.dropna(inplace=True)


Coordinates 47.065025329589844°N 8.341463088989258°E
Elevation 436.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-09,8.915000,0.0,4.155622,345.284851,lucerne
1,2024-11-10,7.227499,0.0,3.710795,349.464783,lucerne
2,2024-11-11,7.452501,6.7,8.019601,345.304749,lucerne
3,2024-11-12,6.335834,3.4,12.744457,35.100521,lucerne
4,2024-11-13,4.737916,1.2,7.596341,19.584793,lucerne
5,2024-11-14,4.065000,0.0,6.112053,355.924774,lucerne
6,2024-11-15,3.269167,0.0,3.710795,293.983429,lucerne
7,2024-11-16,3.840000,0.0,4.044997,330.107605,lucerne
8,2024-11-17,5.333750,0.4,7.100310,27.216135,lucerne


In [74]:
daily_df = daily_df.merge(df_rolling, on='date', how='left')
daily_df.dropna(inplace=True)
daily_df

,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,pm25_rolling_x,pm25_rolling_y,pm25_rolling
0,2024-11-19,9.20,0.1,14.982228,215.217682,lucerne,1.926667,1.526667,1.926667
1,2024-11-20,4.25,0.1,15.542662,256.607483,lucerne,1.093333,1.276667,1.093333


In [9]:
# Insert new data

weather_fg.insert(daily_df)

Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1159319/fs/1150022/fg/1354099


Uploading Dataframe: 100.00% |██████████| Rows 10/10 | Elapsed Time: 00:10 | Remaining Time: 00:00


Launching job: weather_new_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1159319/jobs/named/weather_new_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fe80c3dbbc0>,
 {
   "success": true,
   "statistics": {
     "evaluated_expectations": 2,
     "successful_expectations": 2,
     "unsuccessful_expectations": 0,
     "success_percent": 100.0
   },
   "evaluation_parameters": {},
   "meta": {
     "great_expectations_version": "0.15.12",
     "expectation_suite_name": "weather_expectation_suite",
     "run_id": {
       "run_name": null,
       "run_time": "2024-11-18T22:30:09.339424+00:00"
     },
     "batch_kwargs": {
       "ge_batch_id": "ab240960-a5fc-11ef-8720-6ca100652685"
     },
     "batch_markers": {},
     "batch_parameters": {},
     "validation_time": "20241118T223009.339292Z",
     "expectation_suite_meta": {
       "great_expectations_version": "0.15.12"
     }
   },
   "results": [
     {
       "success": true,
       "expectation_config": {
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": t

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
